In [1]:
from suicide_squad.DataUtils import *
from suicide_squad.DataProcessing import *

In [2]:
squad_to_csv(files,download_folder)

Converting Files to CSV
Finished Downloading and Converting to CSV


['D:\\GitHub\\Suicide-SQuAD\\data\\original\\train-v1.1.csv',
 'D:\\GitHub\\Suicide-SQuAD\\data\\original\\dev-v1.1.csv',
 'D:\\GitHub\\Suicide-SQuAD\\data\\original\\train-v2.0.csv',
 'D:\\GitHub\\Suicide-SQuAD\\data\\original\\dev-v2.0.csv']

In [3]:
dfs=squad_to_df(files,download_folder)

Converting Files to CSV
Finished Downloading and Converting to CSV
Converting Files to DataFrame
Finished Downloading and Converting to DataFrame


In [4]:
dfs.keys()

dict_keys(['train-v1.1', 'dev-v1.1', 'train-v2.0', 'dev-v2.0'])

In [6]:
dfs['train-v1.1']

,doc_text,question_text,answer_start,answer_text,qa_id,is_impossible
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,5733be284776f41900661182,False
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,5733be284776f4190066117f,False
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,5733be284776f41900661180,False
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection,5733be284776f41900661181,False
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary,5733be284776f4190066117e,False
...,...,...,...,...,...,...
87594,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,229,Oregon,5735d259012e2f140011a09d,False
87595,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,414,Rangoon,5735d259012e2f140011a09e,False
87596,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,476,Minsk,5735d259012e2f140011a09f,False
87597,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,199,1975,5735d259012e2f140011a0a0,False
